In [420]:
import requests
import pandas as pd
import numpy as np
import json
import time
import sqlite3
conn = sqlite3.connect('pydatabase')

In [579]:
citybike_df=pd.read_sql('SELECT * FROM den_bike_stations', conn)

In [580]:
citybike_df

,Station_Number,Station_Name,Latitude,Longitude,Number of Bikes
0,0,Walnut St at 26th St,39.759344,-104.986164,0
1,1,Welton St at 15th St,39.743730,-104.992260,4
2,2,12th St at Auraria Pkwy,39.748321,-105.004362,1
3,3,18th St at Wazee St,39.752963,-104.997402,0
4,4,Tremont Pl at Park Ave W,39.750174,-104.980430,2
...,...,...,...,...,...
61,61,Wynkoop St at 17th St,39.752701,-104.999787,0
62,62,Lawrence St at 17th St,39.749613,-104.995105,1
63,63,32nd St at Larimer St,39.764156,-104.977904,0
64,64,Larimer St at 31st St,39.763134,-104.979556,0


In [567]:
def get_fsvenues(latitude, longitude):
    url = "https://api.foursquare.com/v3/places/search"
   
    params = {
        'll' : f"{latitude},{longitude}",
        'radius' : 100,
        'categories' : 13000,
        'fields': 'name,location,rating,distance,popularity',
        'sort':"DISTANCE",
        'limit': 50
    }

    headers = {
        "Accept": "application/json",
        "Authorization":'fsq3zey6wred2fLCHrUka6XF5ZYXNyRIxi07lpgNo8d0CUQ='
    }

    response = requests.get(url, params=params, headers=headers)
    print (response.url)
    print (response.status_code)
    
    return response
pass

In [538]:
fs_results = pd.DataFrame(columns=['Nearest_Station','Foursquare_Results'])

for station in citybike_df['Station_Number']:

    res = get_fsvenues(latitude=citybike_df.iloc[station]['Latitude'], longitude=citybike_df.iloc[station]['Longitude'] )
    rd = res.json()
    new_row = pd.DataFrame( {'Nearest_Station': [station] ,'Foursquare_Results': len(rd['results'] )}  )
    foursquare_results= pd.concat([foursquare_results, new_row], ignore_index=True)

https://api.foursquare.com/v3/places/search?ll=39.759344%2C-104.986164&radius=100&categories=13000&fields=name%2Clocation%2Crating%2Cdistance%2Cpopularity&sort=DISTANCE&limit=50
200
https://api.foursquare.com/v3/places/search?ll=39.74373%2C-104.99226&radius=100&categories=13000&fields=name%2Clocation%2Crating%2Cdistance%2Cpopularity&sort=DISTANCE&limit=50
200
https://api.foursquare.com/v3/places/search?ll=39.748321%2C-105.0043619&radius=100&categories=13000&fields=name%2Clocation%2Crating%2Cdistance%2Cpopularity&sort=DISTANCE&limit=50
200
https://api.foursquare.com/v3/places/search?ll=39.75296271%2C-104.9974017&radius=100&categories=13000&fields=name%2Clocation%2Crating%2Cdistance%2Cpopularity&sort=DISTANCE&limit=50
200
https://api.foursquare.com/v3/places/search?ll=39.7501737%2C-104.9804304&radius=100&categories=13000&fields=name%2Clocation%2Crating%2Cdistance%2Cpopularity&sort=DISTANCE&limit=50
200
https://api.foursquare.com/v3/places/search?ll=39.7496545%2C-104.9930748&radius=100&ca

In [539]:
print(foursquare_results)

    Nearest_Station Foursquare_Results
0                 0                  1
1                 1                  1
2                 2                  1
3                 3                  1
4                 4                  1
..              ...                ...
661              61                 14
662              62                  6
663              63                  4
664              64                  4
665              65                  4

[666 rows x 2 columns]


In [540]:
foursquare_results

,Nearest_Station,Foursquare_Results
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
661,61,14
662,62,6
663,63,4
664,64,4


In [553]:
foursquare_results.to_csv('foursquare.csv', index=False)

In [554]:
import sqlite3
conn = sqlite3.connect('pydatabase')
foursquare_results.to_sql('foursquare', conn, if_exists='replace', index=False)

666

In [568]:
API_KEY = '9x_wkvC8JY9KVIVX5HVRa-DdnO7QJrkvMZgfliwxueKPaGp-FyShTwEAD5KlStb8ziZSqCp58YehntSFR9uVzNuhzGNIDDLhoV3YdvCfAKX05f2bhowoDL8B8xZJZnYx'

def get_yvenues(latitude, longitude):  
    url = 'https://api.yelp.com/v3/businesses/search'

    params = {
            'latitude': latitude,
            'longitude': longitude,
            'radius' : 1000,
            'categories' : 'restaurants,bars',
            'sort_by' : 'rating'
         }

    headers = {
            "accept": "application/json",
            "user-agent": "lhl_projects",
           "Authorization": f'Bearer {API_KEY}'
        }

    response = requests.get(url, params=params, headers=headers)
    return response
    print (response.url)
    print (response.status_code)
pass

In [569]:
i=0
y_results = pd.DataFrame(columns=['Nearest_Station', 'Yelp_Results'])

while i < 100:
    res = get_yvenues(latitude=citybike_df.iloc[station]['Latitude'], longitude=citybike_df.iloc[station]['Longitude'] )
    rd = res.json()
    new_row = pd.DataFrame({'Nearest_Station': [citybike_df.loc[station, 'Station_Number']], 'Yelp_Results': rd['total']})
    yelp_results= pd.concat([yelp_results, new_row], ignore_index=True)

    i=i+1


In [570]:
print(yelp_results)

    Nearest_Station Yelp_Results
0                65           45
1                65           45
2                65           45
3                65           45
4                65           45
..              ...          ...
195              65           45
196              65           45
197              65           45
198              65           45
199              65           45

[200 rows x 2 columns]


In [571]:
yelp_results

,Nearest_Station,Yelp_Results
0,65,45
1,65,45
2,65,45
3,65,45
4,65,45
...,...,...
195,65,45
196,65,45
197,65,45
198,65,45


In [555]:
yelp_results.to_csv('foursquare.csv', index=False)

In [556]:
import sqlite3
conn = sqlite3.connect('pydatabase')
yelp_results.to_sql('yelp', conn, if_exists='replace', index=False)

100

In [551]:
ratings_df = pd.DataFrame(columns = ["Name" ,"Rating"])

In [572]:
import pandas as pd
ratings = {'Name': [], 'Rating': [], 'Nearest_Station':[]}

for i in range (1, 100):
    res = get_yvenues(latitude, longitude)
    rd = res.json()
    z = 0

while z < min(rd['total'], len(rd['businesses'])):
    ratings['Name'].append(rd['businesses'][z]['alias'])
    ratings['Rating'].append(rd['businesses'][z]['rating'])
    
    # Ensure that the index i stays within the valid range of DataFrame citybike_df
    if i < len(citybike_df):
        ratings['Nearest_Station'].append(citybike_df.iloc[i]['Station_Number'])
    
    z = z + 1
    i = i + 1  


In [562]:
print (rd)

{'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}


In [575]:
lengths = {key: len(value) for key, value in ratings.items()}
print(lengths)

{'Name': 20, 'Rating': 20, 'Nearest_Station': 0}


In [576]:
max_length = max(len(lst) for lst in ratings_data.values())

for key in ratings_data:
    while len(ratings_data[key]) < max_length:
        ratings_data[key].append(None)

In [577]:
ratings_df = pd.DataFrame(ratings_data)
ratings_df.sort_values(by='Rating', ascending = False).head(10)

,Name,Rating,Nearest_Station
0,la-sabrosita-denver,5.0,1.0
1209,ta-co-mucho-bueno-denver,5.0,61.0
1100,la-sabrosita-denver,5.0,56.0
1101,20th-street-cafe-denver-4,5.0,56.0
1108,loris-wraps-denver,5.0,56.0
1109,ta-co-mucho-bueno-denver,5.0,56.0
1110,savoy-denver-denver,5.0,56.0
1120,20th-street-cafe-denver-4,5.0,57.0
1121,la-sabrosita-denver,5.0,57.0
1128,loris-wraps-denver,5.0,57.0


In [578]:
ratings_df.to_sql('ratings', conn, if_exists = 'replace', index = False)

2421